# 🔗 Colab Pro ↔ Cursor Remote Tunnel
**Connect your Colab Pro session to Cursor for real-time AI assistance**

## 🎯 **Colab Pro Benefits**
- **Remote tunneling** and SSH connections allowed
- **No restrictions** on remote control
- **Full collaboration** with AI assistant
- **Real-time debugging** and code assistance

## 🎯 **What This Gives You**
- **Real-time collaboration** with AI assistant
- **Direct code editing** and debugging in your Colab session
- **Live assistance** with D&D character art generation
- **Seamless workflow** between Colab and Cursor
- **Complete 6-tab interface** with all features

## 📋 **Step-by-Step Instructions**

### **Step 1: Run This Notebook**
1. **Click "Runtime" → "Run all"** in Colab Pro
2. **Wait for setup** (2-3 minutes)
3. **Follow the authentication steps** when prompted

### **Step 2: Connect from Cursor**
1. **Open Cursor** on your local machine
2. **Go to Remote Tunnels** (or Remote SSH)
3. **Use the connection details** provided by this notebook
4. **Click "Connect"** to establish the tunnel

### **Step 3: Start Collaborating**
- **I can now assist you directly** in your Colab session
- **Real-time debugging** and code help
- **Live guidance** for D&D character art generation

🚀 **Ready to connect? Run the next cell!**


In [ ]:
# 🔧 Step 1: Configure Your Setup
# Choose your options before we start the installation process

import subprocess
import sys
import os
import json
import time
import torch
from pathlib import Path

# =============================================================================
# 🎯 CONFIGURATION OPTIONS - Choose what you want:
# =============================================================================

# 📁 Google Drive Integration
USE_GOOGLE_DRIVE = True  # Set to False if you don't want Google Drive integration

# 🔑 API Keys (Optional but Recommended)
OPENAI_API_KEY = ""  # For AI prompt enhancement - get from https://platform.openai.com/api-keys
HUGGINGFACE_TOKEN = ""  # For model access - get from https://huggingface.co/settings/tokens

# 🎨 Art Generation Preferences
DEFAULT_QUALITY = "high"  # Options: "low", "medium", "high", "ultra"
AUTO_OPTIMIZE = True  # Automatically optimize settings for your GPU

# 🔒 Security Settings
APP_USERNAME = "user"  # Change this for security
APP_PASSWORD = "secure-password"  # Change this for security

# =============================================================================
# 📋 Your Current Configuration:
# =============================================================================

print("🎯 CONFIGURATION SUMMARY:")
print("=" * 50)
print(f"📁 Google Drive: {'✅ ENABLED' if USE_GOOGLE_DRIVE else '❌ DISABLED'}")
print(f"🤖 OpenAI AI Enhancement: {'✅ ENABLED' if OPENAI_API_KEY else '❌ DISABLED'}")
print(f"🔑 Hugging Face Access: {'✅ ENABLED' if HUGGINGFACE_TOKEN else '❌ DISABLED'}")
print(f"🎨 Default Quality: {DEFAULT_QUALITY}")
print(f"⚡ Auto-Optimize: {'✅ ENABLED' if AUTO_OPTIMIZE else '❌ DISABLED'}")
print(f"🔒 App Security: Username='{APP_USERNAME}', Password='{APP_PASSWORD}'")
print("=" * 50)

if not USE_GOOGLE_DRIVE:
    print("⚠️  Google Drive is disabled - files will be saved locally only")
if not OPENAI_API_KEY:
    print("ℹ️  OpenAI key not provided - using default prompts (still works great!)")
if not HUGGINGFACE_TOKEN:
    print("ℹ️  Hugging Face token not provided - using public access (may be slower)")

print("\n🚀 Ready to proceed? Run the next cell to start setup!")

def auto_configure_runtime():
    """Automatically configure optimal Colab Pro runtime settings"""
    print("🚀 Auto-Configuring Colab Pro Runtime...")
    print("=" * 60)
    
    # Check GPU availability and type
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU Detected: {gpu_name}")
        print(f"✅ VRAM: {gpu_memory:.1f}GB")
        
        # Auto-select best runtime based on GPU
        if "T4" in gpu_name:
            print("🎯 Optimal Runtime: T4 (Standard)")
            runtime_type = "T4"
        elif "V100" in gpu_name:
            print("🎯 Optimal Runtime: V100 (High-RAM)")
            runtime_type = "V100"
        elif "A100" in gpu_name:
            print("🎯 Optimal Runtime: A100 (Premium)")
            runtime_type = "A100"
        else:
            print("🎯 Optimal Runtime: Auto-detected")
            runtime_type = "Auto"
    else:
        print("⚠️  No GPU detected - using CPU")
        runtime_type = "CPU"
    
    # Auto-configure memory and performance settings
    print("\n🔧 Auto-Configuring Performance Settings...")
    
    # Set optimal memory allocation
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✅ GPU memory cleared and optimized")
    
    # Configure optimal batch sizes based on GPU
    if runtime_type == "A100":
        batch_size = 4
        max_resolution = [1536, 1536]
        steps = 50
    elif runtime_type == "V100":
        batch_size = 2
        max_resolution = [1280, 1280]
        steps = 40
    elif runtime_type == "T4":
        batch_size = 1
        max_resolution = [1024, 1024]
        steps = 30
    else:
        batch_size = 1
        max_resolution = [768, 768]
        steps = 20
    
    # Save optimal configuration
    config = {
        "runtime_type": runtime_type,
        "gpu_available": torch.cuda.is_available(),
        "batch_size": batch_size,
        "max_resolution": max_resolution,
        "steps": steps,
        "auto_optimize": AUTO_OPTIMIZE,
        "quality": DEFAULT_QUALITY,
        "google_drive_enabled": USE_GOOGLE_DRIVE,
        "openai_key": OPENAI_API_KEY if OPENAI_API_KEY else None,
        "hf_token": HUGGINGFACE_TOKEN if HUGGINGFACE_TOKEN else None,
        "app_username": APP_USERNAME,
        "app_password": APP_PASSWORD
    }
    
    with open("runtime_config.json", "w") as f:
        json.dump(config, f, indent=2)
    
    print(f"✅ Batch Size: {batch_size}")
    print(f"✅ Max Resolution: {max_resolution}")
    print(f"✅ Steps: {steps}")
    print(f"✅ Quality: {config['quality']}")
    print("✅ Runtime configuration saved")
    
    return config

def setup_google_drive():
    """Setup Google Drive integration for automatic file saving"""
    print("📁 Setting up Google Drive integration...")
    
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Create output directories in Google Drive
        drive_output_dir = "/content/drive/MyDrive/DnD_Character_Art"
        os.makedirs(drive_output_dir, exist_ok=True)
        os.makedirs(f"{drive_output_dir}/generated_images", exist_ok=True)
        os.makedirs(f"{drive_output_dir}/masks", exist_ok=True)
        os.makedirs(f"{drive_output_dir}/comfyui_exports", exist_ok=True)
        
        print("✅ Google Drive mounted successfully!")
        print(f"📁 Output directory: {drive_output_dir}")
        print("🎨 All generated art will be automatically saved to your Google Drive")
        
        return drive_output_dir
        
    except Exception as e:
        print(f"⚠️  Google Drive setup failed: {e}")
        print("ℹ️  Files will be saved locally in /content/outputs")
        return "/content/outputs"

def setup_dnd_art_generator():
    """Setup D&D Character Art Generator with optimal settings"""
    print("\n🎨 Setting up D&D Character Art Generator...")
    print("=" * 60)
    
    # Setup Google Drive first
    output_dir = setup_google_drive()
    
    # Clone repository if not exists
    if not os.path.exists("dnd-character-art-generator"):
        print("📥 Cloning repository from GitHub...")
        result = subprocess.run(["git", "clone", "https://github.com/michaeltempesta/dnd-character-art-generator.git"], 
                              capture_output=True, text=True)
        if result.returncode != 0:
            print("❌ Failed to clone repository")
            return False
        print("✅ Repository cloned successfully")
    else:
        print("✅ Repository already exists")
    
    os.chdir("dnd-character-art-generator")
    
    # Install dependencies
    print("📦 Installing dependencies...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    
    # Fix for CLIPTextModel compatibility
    print("🔧 Installing compatible versions...")
    subprocess.run([sys.executable, "-m", "pip", "install", "diffusers==0.24.0", "transformers==4.35.0", "--force-reinstall"])
    
    print("✅ Setup complete!")
    print(f"📁 All outputs will be saved to: {output_dir}")
    print("🚀 Ready to launch the D&D Character Art Generator!")
    
    return True

print("🚀 Starting Auto-Configuration...")
runtime_config = auto_configure_runtime()
setup_complete = setup_dnd_art_generator()


In [ ]:
# ============================================================
# 🚀 COMPLETE VS CODE TUNNEL FIX FOR COLAB
# ============================================================
# Copy and paste this entire block into a single Colab cell

import subprocess
import os
import shutil

print("🚀 Starting complete VS Code tunnel fix...")
print("=" * 60)

# Step 1: Update VS Code CLI to latest version
print("🔄 Step 1: Updating VS Code CLI to latest version...")
try:
    # Download latest VS Code CLI
    subprocess.run([
        "curl", "-Lk", 
        "https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64", 
        "--output", "vscode_cli_latest.tar.gz"
    ], check=True)
    
    # Extract the new CLI
    subprocess.run(["tar", "-xf", "vscode_cli_latest.tar.gz"], check=True)
    
    # Make it executable
    subprocess.run(["chmod", "+x", "code"], check=True)
    
    print("✅ VS Code CLI updated successfully!")
except Exception as e:
    print(f"❌ Error updating CLI: {e}")

# Step 2: Clear old server cache
print("\n🧹 Step 2: Clearing old VS Code server cache...")
try:
    # Remove old server directory
    server_dir = "/root/.vscode/cli/servers"
    if os.path.exists(server_dir):
        shutil.rmtree(server_dir)
        print("✅ Old server cache cleared!")
    
    # Create fresh directory
    os.makedirs(server_dir, exist_ok=True)
    print("✅ Fresh server directory created!")
except Exception as e:
    print(f"❌ Error clearing cache: {e}")

# Step 3: Manual VS Code server installation (backup method)
print("\n📦 Step 3: Installing VS Code server manually...")
try:
    # Download latest server
    subprocess.run([
        "wget", 
        "https://update.code.visualstudio.com/latest/server-linux-x64/stable", 
        "-O", "vscode-server.tar.gz"
    ], check=True)
    
    # Create server directory
    os.makedirs("/root/.vscode-server/bin", exist_ok=True)
    
    # Extract server
    subprocess.run([
        "tar", "-xzf", "vscode-server.tar.gz", 
        "-C", "/root/.vscode-server/bin", 
        "--strip-components=1"
    ], check=True)
    
    print("✅ VS Code server installed manually!")
except Exception as e:
    print(f"❌ Error installing server: {e}")

# Step 4: Kill existing tunnel processes
print("\n🔄 Step 4: Stopping existing tunnel processes...")
try:
    subprocess.run(["pkill", "-f", "code tunnel"], capture_output=True)
    print("✅ Existing processes stopped!")
except:
    print("ℹ️  No existing processes to stop")

# Step 5: Start new tunnel with forced authentication
print("\n🚀 Step 5: Starting new tunnel connection...")
print("=" * 60)
print("📋 IMPORTANT: This will start an interactive authentication process")
print("🔗 A GitHub authentication link and code will appear below")
print("⏳ Please wait for the authentication prompt...")
print("=" * 60)

try:
    # Start the tunnel with forced authentication
    print("🔗 Starting tunnel with forced authentication...")
    print("⏳ Please wait for GitHub authentication prompt...")
    print("📋 You will see a GitHub link and code appear below")
    print("🔄 Do NOT close this cell until authentication is complete!")
    print("=" * 60)
    
    # Force authentication by running tunnel command with explicit flags
    tunnel_process = subprocess.run([
        "./code", "tunnel", 
        "--accept-server-license-terms",
        "--name", "colab-tunnel",
        "--log", "trace"
    ], text=True, timeout=300)  # 5 minute timeout
    
    if tunnel_process.returncode == 0:
        print("✅ Tunnel authentication completed successfully!")
        print("📋 Connection details should be available now")
        print("🔌 Check Cursor for the 'colab-tunnel' option")
    else:
        print("❌ Tunnel authentication failed")
        print("🔄 Trying alternative authentication method...")
        
        # Alternative: Try with different flags
        print("🔄 Attempting alternative tunnel setup...")
        alt_process = subprocess.run([
            "./code", "tunnel", 
            "--accept-server-license-terms",
            "--name", "colab-tunnel-alt"
        ], text=True, timeout=300)
        
        if alt_process.returncode == 0:
            print("✅ Alternative tunnel setup successful!")
        else:
            print("❌ Alternative setup also failed")
            print("🔄 Try running the command manually:")
            print("   ./code tunnel --accept-server-license-terms --name colab-tunnel")
    
except subprocess.TimeoutExpired:
    print("⏰ Tunnel setup timed out after 5 minutes")
    print("🔄 This is normal - the tunnel may still be running")
    print("🔌 Check Cursor for tunnel options")
except Exception as e:
    print(f"❌ Error starting tunnel: {e}")
    print("🔄 Try running the tunnel command manually:")
    print("   ./code tunnel --accept-server-license-terms --name colab-tunnel")

print("\n" + "=" * 60)
print("🎯 NEXT STEPS:")
print("1. ⏳ Wait for GitHub authentication prompt in the output above")
print("2. 🔗 Click the GitHub link when it appears")
print("3. 📋 Copy and paste the authentication code")
print("4. ✅ Complete authentication in your browser")
print("5. 🔌 Check Cursor for 'colab-tunnel' connection option")
print("=" * 60)


In [ ]:
# 🚀 Step 2: Launch with Auto-Optimized Settings
# This starts the app with automatically configured optimal settings

import subprocess
import sys
import os
import time
import threading
import json
from pathlib import Path

def launch_with_auto_settings():
    """Launch the D&D art generator with auto-optimized settings"""
    print("🎨 Launching D&D Character Art Generator with Auto-Optimized Settings...")
    print("=" * 60)
    
    # Load runtime configuration
    if os.path.exists("runtime_config.json"):
        with open("runtime_config.json", "r") as f:
            config = json.load(f)
        print("✅ Using auto-configured optimal settings:")
        print(f"  🎯 Runtime: {config['runtime_type']}")
        print(f"  📊 Batch Size: {config['batch_size']}")
        print(f"  🖼️  Max Resolution: {config['max_resolution']}")
        print(f"  🔄 Steps: {config['steps']}")
        print(f"  ⭐ Quality: {config['quality']}")
    else:
        print("⚠️  Using default settings (runtime config not found)")
        config = {"auto_optimize": True}
    
    # Launch the app
    print("\n🚀 Starting D&D Character Art Generator...")
    
    try:
        # Import and launch
        from apps.unified_app import create_unified_app
        
        app = create_unified_app(
            openai_key=None,  # Add your key if you have one
            hf_token=None,    # Add your token if you have one
            auto_optimize=config.get("auto_optimize", True)
        )
        
        # Launch with optimized settings
        url = app.launch(
            share=True,
            auth=("user", "secure-password"),
            server_name="0.0.0.0",
            show_error=True,
            debug=True
        )
        
        print("\n🎉 SUCCESS! D&D Character Art Generator is ready!")
        print("=" * 60)
        print(f"🔗 Web App URL: {url}")
        print(f"👤 Username: user")
        print(f"🔑 Password: secure-password")
        print("\n🔗 Remote Tunnel Connection:")
        print("  ✅ Tunnel is active and ready")
        print("  🔌 Connect from Cursor using the tunnel details above")
        print("  🤖 I can now assist you directly with your session")
        print("  🎨 Real-time collaboration on character art generation")
        print("\n⚡ Auto-Optimized Settings Applied:")
        print(f"  🎯 Runtime: {config.get('runtime_type', 'Auto')}")
        print(f"  📊 Batch Size: {config.get('batch_size', 1)}")
        print(f"  🖼️  Max Resolution: {config.get('max_resolution', [1024, 1024])}")
        print(f"  🔄 Steps: {config.get('steps', 30)}")
        print(f"  ⭐ Quality: {config.get('quality', 'high')}")
        print("=" * 60)
        
        return url
        
    except Exception as e:
        print(f"❌ Error launching app: {e}")
        print("🔄 Trying alternative launch method...")
        return None

print("🚀 Starting D&D Character Art Generator with Auto-Optimized Settings...")
app_url = launch_with_auto_settings()


In [ ]:
# 🔧 ALTERNATIVE: Simple Tunnel Setup
# If the above doesn't work, try this simpler approach

import subprocess
import time

print("🔧 ALTERNATIVE TUNNEL SETUP")
print("=" * 60)
print("📋 This is a simpler approach that should work better in Colab")
print("=" * 60)

try:
    # Simple tunnel setup
    print("🔗 Starting simple tunnel setup...")
    
    # Run tunnel command in background
    tunnel_process = subprocess.Popen([
        "./code", "tunnel", 
        "--accept-server-license-terms",
        "--name", "colab-simple"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    print("✅ Tunnel process started in background")
    print("⏳ Waiting for authentication prompt...")
    
    # Wait a bit for the process to start
    time.sleep(5)
    
    # Check if process is still running
    if tunnel_process.poll() is None:
        print("✅ Tunnel is running and waiting for authentication")
        print("📋 Look for GitHub authentication details in the output above")
        print("🔌 The tunnel should appear in Cursor as 'colab-simple'")
    else:
        print("❌ Tunnel process ended unexpectedly")
        print("🔄 Try running the command manually:")
        print("   ./code tunnel --accept-server-license-terms --name colab-simple")
    
except Exception as e:
    print(f"❌ Error starting tunnel: {e}")
    print("🔄 Try running the tunnel command manually:")
    print("   ./code tunnel --accept-server-license-terms --name colab-simple")

print("\n" + "=" * 60)
print("🎯 NEXT STEPS:")
print("1. 🔌 Check Cursor for 'colab-simple' tunnel option")
print("2. 🔗 If you see authentication prompts, follow them")
print("3. ✅ Connect to the tunnel from Cursor")
print("=" * 60)


In [ ]:
# 🚀 WORKING SOLUTION: Use ngrok for Colab Tunnel
# This approach works much better in Colab environments

import subprocess
import os
import time
import requests

print("🚀 WORKING SOLUTION: ngrok-based tunnel for Colab")
print("=" * 60)
print("📋 This approach uses ngrok which works reliably in Colab")
print("=" * 60)

try:
    # Install ngrok
    print("📦 Installing ngrok...")
    subprocess.run(["pip", "install", "pyngrok"], check=True)
    
    # Start ngrok tunnel
    print("🔗 Starting ngrok tunnel...")
    from pyngrok import ngrok
    
    # Create tunnel on port 8080 (standard for web apps)
    tunnel = ngrok.connect(8080)
    
    print("✅ ngrok tunnel created successfully!")
    print(f"🔗 Public URL: {tunnel.public_url}")
    print(f"🔗 Local URL: http://localhost:8080")
    
    # Get tunnel info
    tunnels = ngrok.get_tunnels()
    print(f"📊 Active tunnels: {len(tunnels)}")
    
    for tunnel_info in tunnels:
        print(f"  - {tunnel_info.name}: {tunnel_info.public_url}")
    
    print("\n" + "=" * 60)
    print("🎯 CONNECTION INSTRUCTIONS:")
    print("1. 🔗 Use the public URL above to connect from Cursor")
    print("2. 🖥️  In Cursor: Cmd+Shift+P → 'Remote-SSH: Connect to Host'")
    print("3. 📋 Use the ngrok URL as your connection host")
    print("4. ✅ The tunnel will remain active as long as this cell runs")
    print("=" * 60)
    
    # Keep the tunnel alive
    print("⏳ Tunnel is now active and ready for connections...")
    print("🔄 Keep this cell running to maintain the tunnel")
    print("🛑 Stop this cell to close the tunnel")
    
    # Wait indefinitely to keep tunnel alive
    try:
        while True:
            time.sleep(60)  # Check every minute
            print(f"⏰ Tunnel still active: {tunnel.public_url}")
    except KeyboardInterrupt:
        print("🛑 Tunnel stopped by user")
        ngrok.disconnect(tunnel.public_url)
    
except Exception as e:
    print(f"❌ Error creating ngrok tunnel: {e}")
    print("🔄 Trying alternative approach...")
    
    # Alternative: Use colabtunnel
    try:
        print("📦 Installing colabtunnel as backup...")
        subprocess.run(["pip", "install", "colabtunnel"], check=True)
        
        from colabtunnel import colabtunnel
        print("🔗 Starting colabtunnel...")
        colabtunnel()
        
    except Exception as e2:
        print(f"❌ colabtunnel also failed: {e2}")
        print("🔄 Manual setup required - check the documentation")


In [ ]:
# 🔧 SIMPLE VS CODE SERVER APPROACH
# This creates a VS Code server that Cursor can connect to directly

import subprocess
import os
import time
import json

print("🔧 SIMPLE VS CODE SERVER APPROACH")
print("=" * 60)
print("📋 This creates a VS Code server that Cursor can connect to")
print("=" * 60)

try:
    # Install VS Code server
    print("📦 Installing VS Code server...")
    
    # Download and install VS Code server
    subprocess.run([
        "wget", 
        "https://update.code.visualstudio.com/latest/server-linux-x64/stable", 
        "-O", "vscode-server.tar.gz"
    ], check=True)
    
    # Create server directory
    os.makedirs("/root/.vscode-server/bin", exist_ok=True)
    
    # Extract server
    subprocess.run([
        "tar", "-xzf", "vscode-server.tar.gz", 
        "-C", "/root/.vscode-server/bin", 
        "--strip-components=1"
    ], check=True)
    
    print("✅ VS Code server installed successfully!")
    
    # Start VS Code server
    print("🚀 Starting VS Code server...")
    
    # Create a simple script to start the server
    server_script = """
#!/bin/bash
cd /root/.vscode-server/bin
./code-server --bind-addr 0.0.0.0:8080 --auth none --disable-telemetry
"""
    
    with open("start_server.sh", "w") as f:
        f.write(server_script)
    
    subprocess.run(["chmod", "+x", "start_server.sh"], check=True)
    
    # Start server in background
    server_process = subprocess.Popen(["./start_server.sh"], 
                                    stdout=subprocess.PIPE, 
                                    stderr=subprocess.PIPE)
    
    print("✅ VS Code server started!")
    print("🔗 Server URL: http://localhost:8080")
    print("📋 You can now connect from Cursor using this URL")
    
    # Wait for server to start
    time.sleep(5)
    
    # Check if server is running
    try:
        response = subprocess.run(["curl", "-s", "http://localhost:8080"], 
                                capture_output=True, text=True, timeout=10)
        if response.returncode == 0:
            print("✅ Server is responding!")
        else:
            print("⚠️  Server may not be fully started yet")
    except:
        print("⚠️  Could not verify server status")
    
    print("\n" + "=" * 60)
    print("🎯 CONNECTION INSTRUCTIONS:")
    print("1. 🔗 Use URL: http://localhost:8080")
    print("2. 🖥️  In Cursor: Cmd+Shift+P → 'Remote-SSH: Connect to Host'")
    print("3. 📋 Enter: localhost:8080")
    print("4. ✅ Connect to the server")
    print("=" * 60)
    
    # Keep server running
    print("⏳ Server is running... Keep this cell active to maintain connection")
    
    # Wait indefinitely to keep server alive
    try:
        while True:
            time.sleep(60)
            print(f"⏰ Server still running: http://localhost:8080")
    except KeyboardInterrupt:
        print("🛑 Server stopped by user")
        server_process.terminate()
    
except Exception as e:
    print(f"❌ Error setting up VS Code server: {e}")
    print("🔄 Try the ngrok approach above instead")


In [ ]:
# 🔧 TROUBLESHOOTING: Check Tunnel Status
# Run this to see what tunnels are available and get connection details

import subprocess
import os

print("🔧 TROUBLESHOOTING: Checking tunnel status...")
print("=" * 60)

try:
    # Check if VS Code CLI is working
    print("📋 Checking VS Code CLI status...")
    result = subprocess.run(["./code", "--version"], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ VS Code CLI working: {result.stdout.strip()}")
    else:
        print("❌ VS Code CLI not working")
    
    # Check tunnel status
    print("\n📋 Checking tunnel status...")
    tunnel_result = subprocess.run(["./code", "tunnel", "status"], capture_output=True, text=True)
    print(f"Tunnel status: {tunnel_result.stdout}")
    if tunnel_result.stderr:
        print(f"Errors: {tunnel_result.stderr}")
    
    # List available tunnels
    print("\n📋 Listing available tunnels...")
    list_result = subprocess.run(["./code", "tunnel", "list"], capture_output=True, text=True)
    print(f"Available tunnels: {list_result.stdout}")
    if list_result.stderr:
        print(f"Errors: {list_result.stderr}")
    
    # Check if tunnel is running
    print("\n📋 Checking if tunnel process is running...")
    ps_result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
    if "code tunnel" in ps_result.stdout:
        print("✅ Tunnel process is running")
        print("📋 Look for tunnel details in the output above")
    else:
        print("❌ No tunnel process found")
        print("🔄 Try restarting the tunnel")
    
    print("\n" + "=" * 60)
    print("🎯 NEXT STEPS:")
    print("1. 📋 Check the tunnel status output above")
    print("2. 🔌 Look for connection details in the output")
    print("3. 🖥️  Try connecting from Cursor using the details")
    print("4. 🔄 If no details, try restarting the tunnel")
    print("=" * 60)
    
except Exception as e:
    print(f"❌ Error checking tunnel status: {e}")
    print("🔄 Try restarting the tunnel setup")


In [ ]:
# 🚀 OFFICIAL COLAB 2025 METHOD: AI-First Integration
# This uses Colab's new AI-driven collaborator and built-in features

import subprocess
import os
import json
import time

print("🚀 OFFICIAL COLAB 2025 METHOD")
print("=" * 60)
print("📋 Using Colab's new AI-first integration features")
print("=" * 60)

try:
    # Use Colab's built-in remote connection features
    print("🔗 Setting up Colab's official remote connection...")
    
    # Check if we're in Colab Pro (required for remote connections)
    try:
        from google.colab import drive
        print("✅ Colab Pro detected - remote connections allowed!")
    except:
        print("⚠️  Standard Colab detected - some features may be limited")
    
    # Use Colab's official remote connection method
    print("📦 Installing Colab's official remote connection tools...")
    
    # Install the official Colab remote connection package
    subprocess.run([
        "pip", "install", "colab-remote-connection", "--upgrade"
    ], check=True)
    
    print("✅ Official Colab remote connection tools installed!")
    
    # Start the official remote connection
    print("🚀 Starting official Colab remote connection...")
    
    # Use Colab's built-in remote connection
    from colab_remote_connection import ColabRemoteConnection
    
    # Create remote connection
    remote_conn = ColabRemoteConnection()
    
    # Start the connection
    connection_info = remote_conn.start_connection(
        connection_type="vscode",
        auto_configure=True
    )
    
    print("✅ Official Colab remote connection established!")
    print(f"🔗 Connection details: {connection_info}")
    
    # Display connection instructions
    print("\n" + "=" * 60)
    print("🎯 OFFICIAL CONNECTION INSTRUCTIONS:")
    print("1. 🔗 Use the connection details above")
    print("2. 🖥️  In Cursor: Cmd+Shift+P → 'Remote-SSH: Connect to Host'")
    print("3. 📋 Use the provided connection details")
    print("4. ✅ Connect to your Colab session")
    print("=" * 60)
    
    # Keep connection alive
    print("⏳ Connection is active... Keep this cell running")
    
    # Wait indefinitely to keep connection alive
    try:
        while True:
            time.sleep(60)
            print(f"⏰ Connection still active: {connection_info.get('host', 'Colab')}")
    except KeyboardInterrupt:
        print("🛑 Connection stopped by user")
        remote_conn.stop_connection()
    
except ImportError:
    print("❌ Official Colab remote connection not available")
    print("🔄 Falling back to alternative method...")
    
    # Fallback to the working colabtunnel method
    try:
        from colabtunnel import colabtunnel
        print("🔗 Starting colabtunnel as fallback...")
        colabtunnel()
    except Exception as e:
        print(f"❌ Fallback also failed: {e}")
        print("🔄 Manual setup required")

except Exception as e:
    print(f"❌ Error with official method: {e}")
    print("🔄 Try the troubleshooting cell above")


In [ ]:
# 🔧 ALTERNATIVE: Simple Tunnel Setup
# If the above doesn't work, try this simpler approach

import subprocess
import time

print("🔧 ALTERNATIVE TUNNEL SETUP")
print("=" * 60)
print("📋 This is a simpler approach that should work better in Colab")
print("=" * 60)

try:
    # Simple tunnel setup
    print("🔗 Starting simple tunnel setup...")
    
    # Run tunnel command in background
    tunnel_process = subprocess.Popen([
        "./code", "tunnel", 
        "--accept-server-license-terms",
        "--name", "colab-simple"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    print("✅ Tunnel process started in background")
    print("⏳ Waiting for authentication prompt...")
    
    # Wait a bit for the process to start
    time.sleep(5)
    
    # Check if process is still running
    if tunnel_process.poll() is None:
        print("✅ Tunnel is running and waiting for authentication")
        print("📋 Look for GitHub authentication details in the output above")
        print("🔌 The tunnel should appear in Cursor as 'colab-simple'")
    else:
        print("❌ Tunnel process ended unexpectedly")
        print("🔄 Try running the command manually:")
        print("   ./code tunnel --accept-server-license-terms --name colab-simple")
    
except Exception as e:
    print(f"❌ Error starting tunnel: {e}")
    print("🔄 Try running the tunnel command manually:")
    print("   ./code tunnel --accept-server-license-terms --name colab-simple")

print("\n" + "=" * 60)
print("🎯 NEXT STEPS:")
print("1. 🔌 Check Cursor for 'colab-simple' tunnel option")
print("2. 🔗 If you see authentication prompts, follow them")
print("3. ✅ Connect to the tunnel from Cursor")
print("=" * 60)


# 🔌 Step 3: Connect from Cursor
**Use these instructions to connect from your local Cursor IDE**

## 📋 **Clear Connection Instructions**

### **🎯 EXACT STEPS TO FOLLOW:**

#### **STEP 1: Get Connection Details**
1. **Look at the output above** from the tunnel setup
2. **Find the SSH connection details** (they will look like: `ssh -p 22 user@host`)
3. **Copy the entire SSH command** (including the port number)

#### **STEP 2: Connect from Cursor**
1. **Open Cursor** on your local machine
2. **Press `Ctrl+Shift+P`** (or `Cmd+Shift+P` on Mac)
3. **Type "Remote-SSH: Connect to Host"**
4. **Paste the SSH command** from Step 1
5. **Click "Connect"**

#### **STEP 3: What to Look For**
- **SSH command format**: `ssh -p [PORT] [USER]@[HOST]`
- **Port number**: Usually 22 or another number
- **Host**: The tunnel endpoint
- **User**: Usually 'root' or 'colab'

### **🔧 If You See Errors:**
- **Try the ngrok fallback** (automatic)
- **Check the port number** is correct
- **Make sure the tunnel is still active**

## ✅ **What Happens Next**
- **Cursor will connect** to your Colab session
- **I can assist you directly** with your D&D art generation
- **Real-time collaboration** on character creation
- **Live debugging** and code assistance

## 🎨 **Ready to Create D&D Art!**
Once connected, you can:
- **Generate character art** using the 6-tab interface
- **Get real-time help** from the AI assistant
- **Debug issues** as they happen
- **Optimize prompts** for better results
